### from : rag_cot_peactice.md

## 환경 정보

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# .env 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # ex: "gpt-4o-mini"

# LLM 초기화: n개의 샘플 생성
llm = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7,
    # temperature=0.0,
    n=1
)

## AnswerParser

In [2]:
# 세 개의 긴 문서 정의
doc1 = '''현대 교육 분야에서는 **인공지능(AI) 기반 플랫폼**이 빠르게 확산되고 있다. 초기에는 단순히 학습 진도 관리나 자동 채점에 머물렀으나, 최근에는 학습자 개인의 성향과 학습 패턴을 분석해 최적의 학습 경로를 제시하는 **맞춤형 학습 추천 시스템**으로 진화하고 있다. 예를 들어, 학습자의 풀이 시간을 실시간으로 모니터링해 난이도를 조절하고, 오답 유형을 분석해 유사 문제를 자동으로 출제해 주는 기능이 이미 상용화된 상태이다.

또한, **자연어 처리(NLP)** 기술의 발달로 강의 콘텐츠를 자동으로 요약·생성하거나, 학습자의 질문에 실시간으로 답변하는 챗봇 튜터가 등장했다. 이들 챗봇은 단순 키워드 검색을 넘어 의미 기반 검색과 대화형 학습을 지원하여, 교실 수업이나 온라인 강의의 한계를 극복하는 보조 교사 역할을 수행한다.

미래에는 강화 학습 기반 에이전트가 학습자와의 상호작용을 통해 학습 효과를 스스로 최적화하고, **메타러닝** 기법으로 새로운 과목·주제에도 빠르게 적응하는 플랫폼이 나올 것으로 기대된다. 특히, VR·AR 환경과 결합된 몰입형 학습(Immersive Learning)에서는 가상 교실에서 AI 튜터와 1:1 과외를 받는 듯한 경험이 가능해질 전망이다.

이처럼 AI 교육 플랫폼은 교육의 **접근성**, **효율성**, **개인화**를 획기적으로 개선하며**,** 전통적 교실 수업의 패러다임을 재정의하고 있다.'''

doc2 = '''**Retrieval-Augmented Generation(RAG)**은 대규모 언어 모델(LLM)의 생성 능력과 외부 지식 저장소(벡터 DB)를 결합해, 더 정확하고 최신의 정보를 활용한 답변을 생성하는 기술이다. 전통적인 LLM은 학습 시점 이후의 정보나 매우 구체적인 사안에 대해 오류를 범하기 쉽지만, RAG는 벡터 검색을 통해 연관 문서를 찾아와 이를 컨텍스트로 제공함으로써 이러한 한계를 보완한다.

RAG 파이프라인은 주로 세 단계로 이루어진다. 첫째, 사용자의 질의(Query)를 임베딩하여 벡터 DB에서 연관도가 높은 문서를 **Top-K** 방식으로 검색한다. 둘째, 검색된 문서들을 프롬프트에 포함시키고, 셋째, LLM이 이를 종합해 최종 답변을 생성한다. 이 과정에서 문서 정제(cleaning), 메타데이터 필터링, 요약(compression) 등의 전처리 기법이 함께 적용되면 효율성과 정확도가 더욱 향상된다.

실제 활용 사례로는 기업 내부 위키 및 문서 DB를 활용한 **사내 지식 검색 챗봇**, 법률·의학 분야에서 대규모 논문·판례·임상 데이터를 기반으로 한 **전문 상담 시스템**, 그리고 전자상거래 분야에서 고객 리뷰와 상품 설명을 결합해 **정교한 상품 추천**을 제공하는 시스템 등이 있다.

특히, RAG에 **프롬프트 압축(prompt compression)** 기법을 도입하면, LLM 입력 토큰 수를 줄여 응답 비용(cost)을 절감하면서도 핵심 정보를 유지해 답변 품질을 높일 수 있다.'''

doc3 = '''**프롬프트 압축(Prompt Compression)**은 LLM에 입력하는 컨텍스트(문서·대화 로그·검색 결과 등)를 최대한 효율적으로 압축해, 모델 토큰 한도 내에서 핵심 정보만 전달하는 기법이다. 일반적으로 LLM 비용은 입력·출력 토큰 수에 비례하므로, 불필요한 문장이나 중복 정보가 많을수록 비용이 증가하고 응답 속도가 느려진다.

압축 방법은 크게 두 가지로 나뉜다. 첫째, **정적 압축**으로, Python 슬라이싱으로 필요 문구를 잘라내는 방식이다. 둘째, **동적 압축**으로, 요약용 LLM을 별도로 두고 `docs → summarizer → compressed_docs` 순으로 체인을 구성해 의미 기반으로 압축하는 방법이다. 후자는 중요한 세부 사항을 놓치지 않으면서도 더 극적인 토큰 절감 효과를 기대할 수 있다.'''

retrieved_docs = [doc1, doc2, doc3]

### 난이도 하 (Easy)

In [3]:
from langchain.prompts import PromptTemplate
# 두괄식 문장인 경우 압축
trunc = "\n\n".join([" ".join(doc.split()[:200]) for doc in retrieved_docs])    

print(trunc)


현대 교육 분야에서는 **인공지능(AI) 기반 플랫폼**이 빠르게 확산되고 있다. 초기에는 단순히 학습 진도 관리나 자동 채점에 머물렀으나, 최근에는 학습자 개인의 성향과 학습 패턴을 분석해 최적의 학습 경로를 제시하는 **맞춤형 학습 추천 시스템**으로 진화하고 있다. 예를 들어, 학습자의 풀이 시간을 실시간으로 모니터링해 난이도를 조절하고, 오답 유형을 분석해 유사 문제를 자동으로 출제해 주는 기능이 이미 상용화된 상태이다. 또한, **자연어 처리(NLP)** 기술의 발달로 강의 콘텐츠를 자동으로 요약·생성하거나, 학습자의 질문에 실시간으로 답변하는 챗봇 튜터가 등장했다. 이들 챗봇은 단순 키워드 검색을 넘어 의미 기반 검색과 대화형 학습을 지원하여, 교실 수업이나 온라인 강의의 한계를 극복하는 보조 교사 역할을 수행한다. 미래에는 강화 학습 기반 에이전트가 학습자와의 상호작용을 통해 학습 효과를 스스로 최적화하고, **메타러닝** 기법으로 새로운 과목·주제에도 빠르게 적응하는 플랫폼이 나올 것으로 기대된다. 특히, VR·AR 환경과 결합된 몰입형 학습(Immersive Learning)에서는 가상 교실에서 AI 튜터와 1:1 과외를 받는 듯한 경험이 가능해질 전망이다. 이처럼 AI 교육 플랫폼은 교육의 **접근성**, **효율성**, **개인화**를 획기적으로 개선하며**,** 전통적 교실 수업의 패러다임을 재정의하고 있다.

**Retrieval-Augmented Generation(RAG)**은 대규모 언어 모델(LLM)의 생성 능력과 외부 지식 저장소(벡터 DB)를 결합해, 더 정확하고 최신의 정보를 활용한 답변을 생성하는 기술이다. 전통적인 LLM은 학습 시점 이후의 정보나 매우 구체적인 사안에 대해 오류를 범하기 쉽지만, RAG는 벡터 검색을 통해 연관 문서를 찾아와 이를 컨텍스트로 제공함으로써 이러한 한계를 보완한다. RAG 파이프라인은 주로 세 단계로 이루어진다. 첫째, 사용자의 질의(Query)를 임베딩하여 벡터 DB에서 연관도가 높은 문서

In [4]:
template = PromptTemplate.from_template(
    '''
    다음 문서를 읽고 요약하세요. 
    문서:{docs}
    질문: {question}
    '''
)
runnable = template | llm
result = runnable.invoke({'docs': trunc, 'question': '공통 주제는?'})
print(result.content)

공통 주제는 **인공지능(AI) 기술의 교육 및 정보 처리 분야에서의 활용**입니다. 문서에서는 AI 기반 교육 플랫폼의 발전과 맞춤형 학습 추천 시스템, 자연어 처리(NLP) 기술을 활용한 챗봇 튜터의 역할, 그리고 Retrieval-Augmented Generation(RAG) 기술을 통한 정보 검색 및 응답 생성의 효율성을 설명하고 있습니다. 또한, 프롬프트 압축 기법을 통해 LLM의 성능을 최적화하는 방법에 대해서도 다루고 있습니다. 전반적으로 AI 기술이 교육의 개인화, 효율성 및 접근성을 개선하고, 정보 처리에서의 정확성 및 비용 절감을 도모하는 방식에 초점을 맞추고 있습니다.


### 난이도 중 (Medium)

In [5]:
# 요약 기반 동적 압축
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

sum_t = PromptTemplate.from_template('100토큰 이내 요약: {docs}')
summarizer = sum_t | ChatOpenAI(model='gpt-4o-mini', temperature=0.3)
msg = summarizer.invoke({'docs': trunc}); 
summary = msg.content
print(summary)

현대 교육 분야에서는 AI 기반 플랫폼이 개인 맞춤형 학습 추천 시스템으로 발전하고 있으며, 학습자의 성향과 패턴을 분석해 최적의 학습 경로를 제시하고 있다. 자연어 처리 기술을 활용한 챗봇 튜터는 실시간 질문 응답 및 콘텐츠 요약 기능을 제공, 교실 수업의 한계를 극복하고 있다. 미래에는 VR·AR 환경과 결합한 몰입형 학습이 기대된다. 

Retrieval-Augmented Generation(RAG) 기술은 대규모 언어 모델과 외부 지식 저장소를 결합해 정확하고 최신 정보를 제공하며, 프롬프트 압축 기법을 통해 입력 토큰 수를 줄여 비용을 절감하고 응답 품질을 높인다.


In [6]:
qa_t = PromptTemplate.from_template('요약:{summary}\n질문:{question}')
qa = qa_t | ChatOpenAI(model='gpt-4o-mini', temperature=0)
qa_msg  = qa.invoke({'summary': summary, 'question': '공통 주제는?'})
print(qa_msg.content)

공통 주제는 "AI 기술을 활용한 교육 혁신"입니다. AI 기반 플랫폼, 챗봇 튜터, VR·AR 환경, 그리고 RAG 기술 모두 교육 분야에서 학습 경험을 향상시키고 개인 맞춤형 학습을 지원하는 데 중점을 두고 있습니다.


### 난이도 상 (Hard)

In [7]:
# 조건부 압축 및 엄격 JSON 응답
import re, json
from typing import Dict
from langchain_core.runnables.base import Runnable
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

class ConditionalCompressor(Runnable):
    def __init__(self, threshold=300):
        self.threshold = threshold
        self.summarizer = (
            PromptTemplate.from_template('80토큰 요약: {docs}')
            | ChatOpenAI(model='gpt-4o-mini', temperature=0.3)
            | StrOutputParser()
        )
    def invoke(self, inputs: Dict[str, str]) -> str:
        docs = inputs['docs']; words = docs.split()
        if len(words) > self.threshold:
            sliced = ' '.join(words[:self.threshold])
            return self.summarizer.invoke({'docs': sliced})
        return docs

In [8]:
# 1) 압축 수행
docs = '\n\n'.join(retrieved_docs)
compressed_text = ConditionalCompressor().invoke({'docs': docs})
print(compressed_text)


현대 교육에서 AI 기반 플랫폼이 급속히 발전하고 있다. 초기의 자동 채점과 학습 진도 관리에서 벗어나, 개인 맞춤형 학습 추천 시스템으로 진화하고 있으며, 실시간 학습 모니터링과 챗봇 튜터를 통해 학습자의 질문에 즉각 대응하고 있다. 미래에는 VR·AR과 결합한 몰입형 학습이 가능해질 전망이다. 또한, Retrieval-Augmented Generation(RAG) 기술은 대규모 언어 모델과 외부 지식 저장소를 결합해 더 정확한 정보를 제공하며, 다양한 분야에서 활용되고 있다.


In [11]:
# 2) QA with strict JSON only
qa_template = PromptTemplate.from_template(
    '문서:\n{docs}\n질문: {question}\n'
    '응답은 오직 JSON 객체 하나({{"answer": "..."}})만 출력하세요.'
)
qa = qa_template | ChatOpenAI(model='gpt-4o-mini', temperature=0)
qa_msg = qa.invoke({'docs': compressed_text, 'question': '공통 주제는 무엇인가요?'})
print(f'type: {type(qa_msg)}, {qa_msg.content}')


type: <class 'langchain_core.messages.ai.AIMessage'>, {"answer": "AI 기반 교육 기술의 발전과 개인 맞춤형 학습 시스템"}


In [10]:
# 3) 후처리 및 파싱
m = re.search(r"\{.*\}", qa_msg.content, re.S)
if not m:
    raise ValueError('JSON 출력 실패: ' + qa_msg.content)
parsed = json.loads(m.group())
print(f'type: {type(parsed)}, Answer: {parsed["answer"]}')

type: <class 'dict'>, Answer: AI 기반 교육 기술의 발전과 개인 맞춤형 학습 시스템
